In [1]:
!pip install jupyter-black

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 98.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

%load_ext rich
%load_ext jupyter_black

In [3]:
raw_dataset = load_dataset("glue", "mrpc")
raw_dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [4]:
raw_train = raw_dataset["train"]
raw_train[0]


{
    'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
    'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
    'label': 1,
    'idx': 0
}

In [5]:
raw_train.features


{
    'sentence1': Value('string'),
    'sentence2': Value('string'),
    'label': ClassLabel(names=['not_equivalent', 'equivalent']),
    'idx': Value('int32')
}

In [6]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tk1 = tokenizer(raw_train[0]["sentence1"])
tk2 = tokenizer(raw_train[0]["sentence2"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs


{
    'input_ids': [
        101,
        2023,
        2003,
        1996,
        2034,
        6251,
        1012,
        102,
        2023,
        2003,
        1996,
        2117,
        2028,
        1012,
        102
    ],
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
}

In [8]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])


[
    '[CLS]',
    'this',
    'is',
    'the',
    'first',
    'sentence',
    '.',
    '[SEP]',
    'this',
    'is',
    'the',
    'second',
    'one',
    '.',
    '[SEP]'
]

In [9]:
tokenized_dataset = tokenizer(
    raw_dataset["train"]["sentence1"],
    raw_dataset["train"]["sentence2"],
    truncation=True,
    padding=True,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:1                                                                              │
│                                                                                                  │
│ ❱ 1 tokenized_dataset = tokenizer(                                                               │
│   2 │   raw_dataset["train"]["sentence1"],                                                       │
│   3 │   raw_dataset["train"]["sentence2"],                                                       │
│   4 │   truncation=True,                                                                         │
│                                                                                                  │
│ /usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2938 in __call__ │
│                                                                                                  │
│   2935 │   │   │   # input mode in this case.                                                    │
│   2936 │   │   │   if not self._in_target_context_manager:                                       │
│   2937 │   │   │   │   self._switch_to_input_mode()                                              │
│ ❱ 2938 │   │   │   encodings = self._call_one(text=text, text_pair=text_pair, **all_kwargs)      │
│   2939 │   │   if text_target is not None:                                                       │
│   2940 │   │   │   self._switch_to_target_mode()                                                 │
│   2941 │   │   │   target_encodings = self._call_one(text=text_target, text_pair=text_pair_targ  │
│                                                                                                  │
│ /usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2998 in          │
│ _call_one                                                                                        │
│                                                                                                  │
│   2995 │   │   │   │   return False                                                              │
│   2996 │   │                                                                                     │
│   2997 │   │   if not _is_valid_text_input(text):                                                │
│ ❱ 2998 │   │   │   raise ValueError(                                                             │
│   2999 │   │   │   │   "text input must be of type `str` (single example), `list[str]` (batch o  │
│   3000 │   │   │   │   "or `list[list[str]]` (batch of pretokenized examples)."                  │
│   3001 │   │   │   )                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: text input must be of type `str` (single example), `list[str]` (batch or single pretokenized example) 
or `list[list[str]]` (batch of pretokenized examples).

In [10]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [11]:
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [12]:
data_collater = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
samples = tokenized_dataset["train"][:10]
samples = {
    k: v for k, v in samples.items() if k not in ["sentence1", "sentence2", "idx"]
}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32, 45, 60]

In [18]:
batch = data_collater(samples)
{k: v.shape for k, v in batch.items()}


{
    'input_ids': torch.Size([10, 67]),
    'token_type_ids': torch.Size([10, 67]),
    'attention_mask': torch.Size([10, 67]),
    'labels': torch.Size([10])
}

In [22]:
get_dataset_config_info("glue", "mrpc")


DatasetInfo(
    description='',
    citation='',
    homepage='',
    license='',
    features={
        'sentence1': Value('string'),
        'sentence2': Value('string'),
        'label': ClassLabel(names=['not_equivalent', 'equivalent']),
        'idx': Value('int32')
    },
    post_processed=None,
    supervised_keys=None,
    builder_name='parquet',
    dataset_name='glue',
    config_name='mrpc',
    version=0.0.0,
    splits={
        'train': SplitInfo(
            name='train',
            num_bytes=944761,
            num_examples=3668,
            shard_lengths=None,
            dataset_name='glue'
        ),
        'validation': SplitInfo(
            name='validation',
            num_bytes=105981,
            num_examples=408,
            shard_lengths=None,
            dataset_name='glue'
        ),
        'test': SplitInfo(
            name='test',
            num_bytes=442842,
            num_examples=1725,
            shard_lengths=None,
            dataset_name=

In [20]:
from datasets import get_dataset_config_info

get_dataset_config_info("glue", "sst2")


DatasetInfo(
    description='',
    citation='',
    homepage='',
    license='',
    features={
        'sentence': Value('string'),
        'label': ClassLabel(names=['negative', 'positive']),
        'idx': Value('int32')
    },
    post_processed=None,
    supervised_keys=None,
    builder_name='parquet',
    dataset_name='glue',
    config_name='sst2',
    version=0.0.0,
    splits={
        'train': SplitInfo(
            name='train',
            num_bytes=4681603,
            num_examples=67349,
            shard_lengths=None,
            dataset_name=None
        ),
        'validation': SplitInfo(
            name='validation',
            num_bytes=106252,
            num_examples=872,
            shard_lengths=None,
            dataset_name=None
        ),
        'test': SplitInfo(
            name='test',
            num_bytes=216640,
            num_examples=1821,
            shard_lengths=None,
            dataset_name=None
        )
    },
    download_checksums=None

#### Suggested Exercise

In [28]:
sst_dataset = load_dataset("glue", "sst2")
sst_dataset


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [29]:
sst_dataset["train"][:10]


{
    'sentence': [
        'hide new secretions from the parental units ',
        'contains no wit , only labored gags ',
        'that loves its characters and communicates something rather beautiful about human nature ',
        'remains utterly satisfied to remain the same throughout ',
        'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ',
        "that 's far too tragic to merit such superficial treatment ",
        'demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . ',
        'of saucy ',
        "a depressed fifteen-year-old 's suicidal poetry ",
        "are more deeply thought through than in most ` right-thinking ' films "
    ],
    'label': [0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
    'idx': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
}

In [26]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [27]:
def tokenize_sst(sentence):
    return tokenizer(sentence["sentence"], truncation=True)

In [31]:
tokenized_dataset = sst_dataset.map(tokenize_sst, batched=True)
tokenized_dataset

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})